In [1]:
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn import linear_model
import import_ipynb
from Garrett_notebook import scatter_graph
from Mike_notebook import bubble1
from Mike_notebook import bubble2

importing Jupyter notebook from Garrett_notebook.ipynb
importing Jupyter notebook from Mike_notebook.ipynb


In [2]:
## Import all clean data
gov_corruption_df = pd.read_csv('clean data/cpi_2019.csv')
generosity_df = pd.read_csv('clean data/global_generosity_score.csv')
human_freedom_df = pd.read_csv('clean data/hf_score.csv')
life_expectancy_df = pd.read_csv('clean data/lifeexp.csv')
education_df = pd.read_csv('clean data/clean_edu.csv')
gdp_df = pd.read_csv('clean data/Updated_GDP.csv')
happy_df = pd.read_csv('clean data/happy_score.csv')

In [3]:
## Clean up happy score dataframe for merge
happy_df = happy_df.rename(columns={'Country or region':'Country','Score':'Happiness Score'})
#happy_df

In [4]:
## Clean up generosity dataframe for merge
generosity_df = generosity_df.drop('Unnamed: 0',axis=1)
clean_gen_df = generosity_df.rename(columns={'country':'Country','overallScore':'Generosity Score'})
clean_gen_df = clean_gen_df.sort_values('Country')
clean_gen_df = clean_gen_df.reset_index(drop=True)
#clean_gen_df

In [5]:
## Clean up corruption dataframe for merge
clean_corrupt_df = gov_corruption_df.sort_values('Country')
clean_corrupt_df = clean_corrupt_df.reset_index(drop=True)
#clean_corrupt_df

In [6]:
## Clean up gdp dataframe for merge
gdp_df = gdp_df.rename(columns={'Country Name':'Country','Country Code':'Region','2019':"GDP (in 100s)"})
gdp_df = gdp_df[['Country',"GDP (in 100s)"]]
#gdp_df


In [7]:
## Clean up freedom dataframe for merge
human_freedom_df = human_freedom_df.rename(columns={'countries':'Country','region':'Region','hf_score':'Human Freedom Score'})
#human_freedom_df

In [8]:
## Clean up education dataframe for merge
education_df = education_df.rename(columns={'Data Source':'Country','Rate':'Education'})
#education_df

In [9]:
## merge dataframes into final data frame for analysis
merge_1 = pd.merge(clean_corrupt_df,clean_gen_df,how='outer',on='Country')
merge_2 = pd.merge(merge_1,human_freedom_df,how='outer',on='Country')
merge_3 = pd.merge(merge_2,life_expectancy_df,how='outer',on='Country')
merge_4 = pd.merge(merge_3,education_df,how='outer',on='Country')
merge_5 = pd.merge(merge_4,gdp_df,how='outer',on='Country')
final_data_df = pd.merge(merge_5,happy_df,how='outer',on='Country')


In [10]:
## rename dataframe
final_data_df = final_data_df[['Country','Region','Happiness Score','CPI score 2019','Generosity Score',
                               'Human Freedom Score','Life Expectancy','Education','GDP (in 100s)']]
final_data_df = final_data_df.dropna(axis=0,how='any')
final_data_df = final_data_df.reset_index(drop=True)

In [11]:
#final_data_df['Education'] = final_data_df['Education'].apply(lambda x: "{:.2f}%".format((x/1)))
#final_data_df['GDP (in 100s)'] = final_data_df['GDP (in 100s)'].apply(lambda x: "${:.2f}".format((x/1000)))
final_data_df

,Country,Region,Happiness Score,CPI score 2019,Generosity Score,Human Freedom Score,Life Expectancy,Education,GDP (in 100s)
0,Albania,ECA,4.719,35.0,23.0,7.81,78.0,94.153337,5353.244856
1,Argentina,AME,6.086,45.0,28.0,7.05,76.6,89.302147,9912.281809
2,Armenia,ECA,4.559,42.0,21.0,7.69,76.0,90.511539,4622.733493
3,Austria,WE/EU,7.246,77.0,45.0,8.45,81.6,97.130831,50137.662776
4,Azerbaijan,ECA,5.208,30.0,24.0,6.29,71.4,86.941317,4793.587020
...,...,...,...,...,...,...,...,...,...
83,Ukraine,ECA,4.332,30.0,24.0,6.45,73.0,96.159267,3659.031312
84,United Arab Emirates,MENA,6.825,71.0,45.0,6.17,76.1,78.312130,43103.323058
85,United States,AME,6.892,69.0,58.0,8.44,78.5,104.118214,65297.517508
86,Zambia,SSA,4.107,34.0,39.0,6.71,62.5,54.768871,1305.063254


In [ ]:
## Locate specific Country
#usa = final_data_df.loc[final_data_df['Country']=='United States']
#usa

In [ ]:
summary_df = final_data_df.describe()
summary_df

In [ ]:
region_group_df = final_data_df.groupby('Region').mean()
#region_group_df

In [ ]:
## Multiple Regresion Models


y = final_data_df['Happiness Score']
x = final_data_df[['CPI score 2019','Generosity Score','Human Freedom Score','Life Expectancy','Education','GDP (in 100s)']]

regr = linear_model.LinearRegression()
regr.fit(x,y)

print(regr.coef_)



In [ ]:
plt.style.use('Solarize_Light2')
#print(plt.style.available)

In [ ]:
x=final_data_df['CPI score 2019']
y=final_data_df['Happiness Score']

scatter_graph(x,y)

In [ ]:
x=final_data_df['Generosity Score']
y=final_data_df['Happiness Score']

scatter_graph(x,y)

In [ ]:
x=final_data_df['Human Freedom Score']
y=final_data_df['Happiness Score']

scatter_graph(x,y)

In [ ]:
x=final_data_df['Life Expectancy']
y=final_data_df['Happiness Score']

scatter_graph(x,y)

In [ ]:
x=final_data_df['Education']
y=final_data_df['Happiness Score']

scatter_graph(x,y)


In [ ]:
x=final_data_df['GDP (in 100s)']
y=final_data_df['Happiness Score']

scatter_graph(x,y)

In [14]:
bubble1()

NameError: name 'final_data_df' is not defined

In [ ]:
bubble2()